In [2]:
!pip install -r requirement.txt
!pip install pydub

  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached astor-0.8.1-py2.py3-none-any.whl (27 kB)
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached grpcio-1.54.0-cp38-cp38-win_amd64.whl (4.1 MB)
  Using cached h5py-3.8.0-cp38-cp38-win_amd64.whl (2.7 MB)
  Using cached ipython_genutils-0.2.0-py2.py3-none-any.whl (26 kB)
  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Using cached Keras_Applications-1.0.8-py3-none-any.whl (50 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached Markdown-3.4.3-py3-none-any.whl (93 kB)
  Using cached numpy-1.24.3-cp38-cp38-win_amd64.whl (14.9 MB)
  Using cached opencv_python-4.7.0.72-cp37-abi3-win_amd64.whl (38.2 MB)
  Using cached pexpect-4.8.0-py2.py3-none-any.whl (59 kB)
  Using cached protobuf-4.22.3-cp38-cp38-win_amd64.whl (420 kB)
  Using cached ptyprocess-0.7.0-py2.py3-none-any.whl (13 kB)
  Using cached PyAudio-0.2

In [3]:
import tensorflow as tf
import numpy as np
import os
import glob
import pickle
import pyaudio
import time
from numpy import genfromtxt
from keras import backend as K
from keras.models import load_model
import sys
    
K.set_image_data_format('channels_first')
# np.set_printoptions(threshold=np.nan)
np.set_printoptions(threshold=sys.maxsize)

import pyaudio
from IPython.display import Audio, display, clear_output
import wave
from scipy.io.wavfile import read

from sklearn.mixture import GaussianMixture as GMM
# from sklearn.mixture import GMM 
# from sklearn import mixture

import warnings

warnings.filterwarnings("ignore")

from sklearn import preprocessing
# for converting audio to mfcc
import python_speech_features as mfcc

In [6]:
def calculate_delta(array):
    rows,cols = array.shape
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j < 0:
                first = 0
            else:
                first = i-j
            if i+j > rows -1:
                second = rows -1
            else:
                second = i+j
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas

#convert audio to mfcc features
def extract_features(audio,rate):    
    mfcc_feat = mfcc.mfcc(audio,rate, 0.025, 0.01,20,appendEnergy = True, nfft=1400)
    mfcc_feat = preprocessing.scale(mfcc_feat)
    delta = calculate_delta(mfcc_feat)

    #combining both mfcc features and delta
    combined = np.hstack((mfcc_feat,delta)) 
    return combined

In [7]:
def add_user():
    
    name = input("Enter Name:")
     # check for existing database
    if os.path.exists('./voice_database/embeddings.pickle'):
        with open('./voice_database/embeddings.pickle', 'rb') as database:
            db = pickle.load(database)   
            
            if name in db:
                print("Name Already Exists! Try Another Name...")
                return
    else:
        #if database not exists than creating new database
        db = {}


    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    RATE = 44100
    CHUNK = 1024
    RECORD_SECONDS = 10

    source = "./voice_database/" + name


    os.mkdir(source)

    for i in range(3):
        audio = pyaudio.PyAudio()

        if i == 0:
            j = 3
            while j>=0:
                time.sleep(1.0)
                print("Speak your Aadhar number in {} seconds".format(j))
                clear_output(wait=True)

                j-=1

        elif i ==1:
            print("One more time")
            time.sleep(0.5)

        else:
            print("One last time")
            time.sleep(0.5)

        # start Recording
        stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True,
                    frames_per_buffer=CHUNK)

        print("recording...")
        frames = []

        for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
            data = stream.read(CHUNK)
            frames.append(data)

        # stop Recording
        stream.stop_stream()
        stream.close()
        audio.terminate()

        # saving wav file of speaker
        waveFile = wave.open(source + '/' + str((i+1)) + '.wav', 'wb')
        waveFile.setnchannels(CHANNELS)
        waveFile.setsampwidth(audio.get_sample_size(FORMAT))
        waveFile.setframerate(RATE)
        waveFile.writeframes(b''.join(frames))
        waveFile.close()
        print("Done")

    dest =  "./gmm_models/"
    count = 1

    for path in os.listdir(source):
        path = os.path.join(source, path)

        features = np.array([])

        # reading audio files of speaker
        (sr, audio) = read(path)

        # extract 40 dimensional MFCC & delta MFCC features
        vector   = extract_features(audio,sr)

        if features.size == 0:
            features = vector
        else:
            features = np.vstack((features, vector))

        # when features of 3 files of speaker are concatenated, then do model training
        if count == 3:    
            gmm = GMM(n_components = 16, max_iter=200,covariance_type='diag',n_init = 33)
            gmm.fit(features)

            # saving the trained gaussian model
            pickle.dump(gmm, open(dest + name + '.GMM', 'wb'))
            print(name + ' added successfully') 

            features = np.asarray(())
            count = 0
        count = count + 1

In [8]:
name = 'unknown_test'
dest =  "./gmm_models/"
count = 1
source = "./voice_database/" + name

for path in os.listdir(source):
        path = os.path.join(source, path)

        features = np.array([])

        # reading audio files of speaker
        (sr, audio) = read(path)
        

        # extract 40 dimensional MFCC & delta MFCC features
        vector   = extract_features(audio,sr)
        print(count)

        if features.size == 0:
            features = vector
        else:
            features = np.vstack((features, vector))

        # when features of 3 files of speaker are concatenated, then do model training
        if count == 33:    
            gmm = GMM(n_components = 16, max_iter=200,covariance_type='diag',n_init = 33)
            gmm.fit(features)
            print('*')

            # saving the trained gaussian model
            
            pickle.dump(gmm, open(dest + name + '.GMM', 'wb'))
            print(name + ' added successfully') 

            features = np.asarray(())
            count = 0
        count = count + 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
*
unknown_test added successfully


In [9]:
add_user()

recording...
Done
One more time
recording...
Done
One last time
recording...
Done
Swaroop added successfully


In [11]:
def recognize():
    # Voice Authentication
    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    RATE = 44100
    CHUNK = 1024
    RECORD_SECONDS = 10
    FILENAME = "./test.wav"
    audio = pyaudio.PyAudio()
    # start Recording
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True,
                    frames_per_buffer=CHUNK)
    print("recording...")
    frames = []
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    print("finished recording")
    # stop Recording
    stream.stop_stream()
    stream.close()
    audio.terminate()
    # saving wav file 
    waveFile = wave.open(FILENAME, 'wb')
    waveFile.setnchannels(CHANNELS)
    waveFile.setsampwidth(audio.get_sample_size(FORMAT))
    waveFile.setframerate(RATE)
    waveFile.writeframes(b''.join(frames))
    waveFile.close()
    modelpath = "./gmm_models/"
    gmm_files = [os.path.join(modelpath,fname) for fname in 
                os.listdir(modelpath) if fname.endswith('.GMM')]
    models    = [pickle.load(open(fname,'rb')) for fname in gmm_files]
    speakers   = [fname.split("/")[-1].split(".GMM")[0] for fname 
                in gmm_files]
    
    if len(models) == 0:
        print("No Users Authorized!")
        return
        
    #read test file
    sr,audio = read(FILENAME)
    # extract mfcc features
    vector = extract_features(audio,sr)
    log_likelihood = np.zeros(len(models)) 
    #checking with each model one by one
    for i in range(len(models)):
        gmm = models[i]         
        scores = np.array(gmm.score(vector))
        log_likelihood[i] = scores.sum()
    
    pred = np.argmax(log_likelihood)
    print(pred)
    print(speakers)
    identity = speakers[pred]
    print(identity)
    print(log_likelihood)
    # if voice not recognized than terminate the process
#     print(speakers)
    if identity == 'unknown_test' and 'unknown':
        print("Not Recognized! Try again...")
    elif identity == 'blank':
        print("No voice detected!")
    else:
        print( "Recognized as - ", identity)

In [12]:
recognize()

recording...
finished recording
6
['anushka', 'blank', 'jaya maam', 'new2', 'shwetha maam', 'srivatsa', 'Swaroop', 'unknown', 'unknown_test']
Swaroop
[-22.94055675 -28.69480969 -24.93155755 -21.8854966  -24.80655784
 -24.97940369 -19.11337366 -51.78340956 -22.19023216]
Recognized as -  Swaroop
